In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [29]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime

headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }
base_url = 'https://www.salzburgerfestspiele.at/language/en-us/institution/archiv/archiv-suchergebnisse?dv=1.1.1900&db=31.12.2018&typ=1'

In [30]:
listings = []

r  = requests.get(base_url, headers=headers)
soup = BeautifulSoup(r.text, "html5lib")
results = soup.find('div', attrs={'class': 'ModsuiteArchiveSearchResultC'}).find_all('a')

for result in results:
    work = result.get_text().strip()
    work_url = result.get('href')
    r  = requests.get(work_url, headers=headers)
    soup = BeautifulSoup(r.text, "html5lib")
    
    try:
        composer = soup.find('h1', attrs={'class': 'A_Oper'}).find('span').previous_sibling.strip()
    except:
        composer = ''        
        
    try:
        performances = soup.find('div', attrs={'class': 'DIV_serviceSquare seconRowservice autoheight'}).find_all('li')
    except:
        performances = []
        
    for performance in performances:
        date = performance.get_text().replace('(CANCELLED)', '').strip()
        try:
            date = datetime.strptime(date, '%d %B %Y, %H:%M').date()
        except:
            date = datetime.strptime(date, '%d %B %Y').date()            
        listing = []
        listing.append(work)                        
        listing.append(composer)
        listing.append(date)
#         print(listing)
        listings.append(listing)

#Saving results into dataframe and folder            
import pandas as pd                     
headers = ['work', 'composer', 'date']
df = pd.DataFrame(listings, columns=headers)
df.to_csv('../../data/processed/listings/salzburg.csv')                                    